In [1]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import requests
import json
import datetime
import tweepy

In [2]:
df = pd.read_csv('llavero.csv')
df

,consumer_key,consumer_secret,bearer_token,access_token,access_token_secret,coinmarketcap
0,qTi2G4s8zuybhF5LReyIGvQpx,a0nj2L7DRz3LktHhei4nG7y4ymNnDD7LiIqQmLnw3thzN4...,AAAAAAAAAAAAAAAAAAAAAJkHNQEAAAAAqsEpKXKU8XlNgr...,740937600475795456-rDP7E53ht8AalwPMgNMYohUv4VF...,yETFBKuwYcKxt6a246GeBOBsNCYBVc46aV5wqzEt3Tx7D,71e8ae85-d1b5-45e9-a6e7-fa65d58fabcc


In [3]:
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        df['consumer_key'][0],
        'consumer_secret':     df['consumer_secret'][0],
        'access_token_key':    df['access_token'][0],
        'access_token_secret': df['access_token_secret'][0]
    }
    
    # Credenciales, creo que no tiene un efecto en el resultado
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user='FernGaminho7', # USER!!!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [4]:
miner=TweetMiner(result_limit = 200 )

In [9]:
mined_tweets = miner.mine_user_tweets(user='elonmusk', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [10]:
mined_tweets_df.shape

(1801, 15)